**Трансформация датасета**

json-файлы -> формат, подходящий для YOLO

In [ ]:
import os
import shutil
import json

# Указываем путь к папке в Google Colab где лежат файлы с разметкой
folder_path = '/content/ann'

# Получаем список файлов в папке
files = os.listdir(folder_path)

# Проходимся по каждому файлу
for file_name in files:

    #открываем старый фаил и считываем данные
    file_path = os.path.join(folder_path, file_name)
    json_file = open(file_path, 'r')
    data = json.load(json_file)

    #создаем txt файл куда будем записывать новый формат
    file_name = file_path.split('/')[-1][:-9] + '.txt'
    text_file = open(file_name, 'w')

    #сохраяем ширину и высоту фото
    image_height = data['size']['height']
    image_width = data['size']['width']

    #проходимся по каждому объкту на фото
    for i in range(len(data['objects'])):

        #считаем координаты в нужном формате
        x1 = data['objects'][i]['points']['exterior'][0][0]
        y1 = data['objects'][i]['points']['exterior'][0][1]
        x2 = data['objects'][i]['points']['exterior'][1][0]
        y2 = data['objects'][i]['points']['exterior'][1][1]
        x_center = ((x1 + x2)/2) / image_width
        y_center = ((y1 + y2)/2) / image_height
        rel_width = (x2 - x1) / image_width
        rel_height = (y2 - y1) / image_height
        new_row = '0 ' + str(x_center) + ' ' + str(y_center) + ' '+ str(rel_width) + ' ' + str(rel_height) + '\n'

        #записываем координаты на новую строку текстового файла
        text_file.write(new_row)

    #сохраняем и закрываем оба файла
    text_file.close()
    json_file.close()

**Аугментация датасета**

Делаем 5 преобразований:
1. Зашумление
2. Увеличение яркости
3. Уменьшение яркости
4. Увеличение насыщенности
5. Уменьшение насыщенности

In [1]:
from google.colab import drive
import os

drive.mount('/content/gdrive')

Mounted at /content/gdrive


Создание и сохранение изображений

In [ ]:
import cv2
import numpy as np
from skimage.util import random_noise
import os

# Функция для изменения яркости и насыщенности изображения
def adjust_brightness_and_saturation(image, brightness_factor, saturation_factor):
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    hsv[:, :, 2] = np.clip(hsv[:, :, 2] * brightness_factor, 0, 255)
    hsv[:, :, 1] = np.clip(hsv[:, :, 1] * saturation_factor, 0, 255)
    new_image = cv2.cvtColor(hsv, cv2.COLOR_HSV2BGR)
    return new_image

input_folder = "./gdrive/MyDrive/yolov7_run/images"
train_output_folder = "./gdrive/MyDrive/yolov7_run/yolov7/data/train/images"
test_output_folder = "./gdrive/MyDrive/yolov7_run/yolov7/data/val/images"

test_idx = np.random.randint(0, 930, size=100)
test_names = []

# Проходим по всем изображениям в папке
i = 1
for filename in os.listdir(input_folder):
    if filename.endswith(".png"):

        image = cv2.imread(os.path.join(input_folder, filename))

        #оригинальное изображение
        output_folder = train_output_folder
        if i in test_idx:
            output_folder = test_output_folder
            test_names.append(filename)
        cv2.imwrite(os.path.join(output_folder, f"{os.path.splitext(filename)[0]}.png"), image)

        # повышенная яркость
        i += 1
        brightened_image = adjust_brightness_and_saturation(image, brightness_factor=1.5, saturation_factor=1.2)
        output_folder = train_output_folder
        if i in test_idx:
            output_folder = test_output_folder
            test_names.append(filename)
        cv2.imwrite(os.path.join(output_folder, f"{os.path.splitext(filename)[0]}_br.png"), brightened_image)

        # пониженная яркость
        i += 1
        brightened_image_1 = adjust_brightness_and_saturation(image, brightness_factor=0.7, saturation_factor=1.2)
        output_folder = train_output_folder
        if i in test_idx:
            output_folder = test_output_folder
            test_names.append(filename)
        cv2.imwrite(os.path.join(output_folder, f"{os.path.splitext(filename)[0]}_br1.png"), brightened_image_1)

        # повышенная насыщенность
        i += 1
        saturated_image = adjust_brightness_and_saturation(image, brightness_factor=1.0, saturation_factor=2.0)
        output_folder = train_output_folder
        if i in test_idx:
            output_folder = test_output_folder
            test_names.append(filename)
        cv2.imwrite(os.path.join(output_folder, f"{os.path.splitext(filename)[0]}_sat.png"), saturated_image)

        # защумленность
        i += 1
        noisy_image = random_noise(image, mode='s&p', amount=0.2)
        noisy_image = (255 * noisy_image).astype(np.uint8)
        output_folder = train_output_folder
        if i in test_idx:
            output_folder = test_output_folder
            test_names.append(filename)
        cv2.imwrite(os.path.join(output_folder, f"{os.path.splitext(filename)[0]}_noisy.png"), noisy_image)

        i += 1

Создание и сохранение разметки

In [9]:
test_names_cp = os.listdir('./gdrive/MyDrive/yolov7_run/yolov7/data/val/images')
test_names_cp = [i.split('.')[0] for i in test_names_cp]

def write_txt(frm, to):
    with open(frm, 'r') as source_file:
        file_content = source_file.read()
    with open(to, 'w+') as new_file:
        new_file.write(file_content)

label_input_folder = "./gdrive/MyDrive/yolov7_run/labels"
label_output_train = './gdrive/MyDrive/yolov7_run/yolov7/data/train/labels'
label_output_test = './gdrive/MyDrive/yolov7_run/yolov7/data/val/labels'

for filename in os.listdir(label_input_folder):
    #оригинальное изображение
    output_dir =  label_output_train
    if filename.split('.')[0] in test_names_cp:
        output_dir = label_output_test
    # print(filename, output_dir)
    write_txt(label_input_folder + '/' + filename, output_dir + '/' + filename)

    #повышенная яркость
    output_dir =  label_output_train
    new_filename1 = filename.split('.')[0] + '_br.txt'
    if new_filename1.split('.')[0] in test_names_cp:
        output_dir = label_output_test
    # print(new_filename1, output_dir)
    write_txt(label_input_folder + '/' + filename, output_dir + '/' + new_filename1)

    #пониженная яркость
    output_dir =  label_output_train
    new_filename2 = filename.split('.')[0] + '_br1.txt'
    if new_filename2.split('.')[0] in test_names_cp:
        output_dir = label_output_test
    # print(new_filename2, output_dir)
    write_txt(label_input_folder + '/' + filename, output_dir + '/' + new_filename2)

    #повышенная насыщенность
    output_dir =  label_output_train
    new_filename3 = filename.split('.')[0] + '_sat.txt'
    if new_filename3.split('.')[0] in test_names_cp:
        output_dir = label_output_test
    # print(new_filename3, output_dir)
    write_txt(label_input_folder + '/' + filename, output_dir + '/' + new_filename3)

    #зашумленность
    output_dir =  label_output_train
    new_filename4 = filename.split('.')[0] + '_noisy.txt'
    if new_filename4.split('.')[0] in test_names_cp:
        output_dir = label_output_test
    # print(new_filename4, output_dir)
    write_txt(label_input_folder + '/' + filename, output_dir + '/' + new_filename4)


In [10]:
assert len(os.listdir("./gdrive/MyDrive/yolov7_run/yolov7/data/train/images")) == len(os.listdir("./gdrive/MyDrive/yolov7_run/yolov7/data/train/labels"))
assert len(os.listdir("./gdrive/MyDrive/yolov7_run/yolov7/data/val/images")) == len(os.listdir("./gdrive/MyDrive/yolov7_run/yolov7/data/val/labels"))